In [1]:
# %% [markdown]
# # 🗂️ Indexation FAISS des Offres d'Emploi
# 
# **Objectif** : Créer un index FAISS pour recherche rapide et scalable
# 
# **Pipeline** :
# 1. Charger les offres depuis `jobs_dataset.json`
# 2. Créer l'index FAISS avec embeddings
# 3. Tester la recherche
# 4. Sauvegarder l'index

# %%
import sys
sys.path.append('..')

import json
from pathlib import Path
from src.vector_store import JobVectorStore

# %%
# 1. CHARGEMENT DES OFFRES
print("="*60)
print("📂 CHARGEMENT DES OFFRES")
print("="*60)

jobs_path = Path('../data/jobs/jobs_dataset.json')

if not jobs_path.exists():
    raise FileNotFoundError(
        f"Dataset d'offres introuvable : {jobs_path}\n"
        "Exécutez d'abord le notebook 04_job_generation.ipynb"
    )

with open(jobs_path, 'r', encoding='utf-8') as f:
    dataset = json.load(f)
    jobs = dataset['jobs']

print(f"\n✅ {len(jobs)} offres chargées")
print(f"📊 Catégories : {set(job['category'] for job in jobs)}")

# Afficher un exemple
print(f"\n📄 Exemple d'offre :")
example = jobs[0]
print(f"   ID : {example['job_id']}")
print(f"   Titre : {example['title']}")
print(f"   Entreprise : {example['company']}")
print(f"   Description : {example['description'][:100]}...")
print(f"   Requirements : {example['requirements'][:3]}...")

📂 CHARGEMENT DES OFFRES

✅ 25 offres chargées
📊 Catégories : {'frontend_developer', 'python_developer', 'ml_engineer', 'devops_engineer', 'data_scientist'}

📄 Exemple d'offre :
   ID : job_001
   Titre : Junior ML Engineer
   Entreprise : AI Startup Paris
   Description : Nous recherchons un Junior ML Engineer passionné pour rejoindre notre équipe R&D.

Responsabilités :...
   Requirements : ['Python (numpy, pandas, scikit-learn)', 'Machine Learning basics (supervised learning)', 'Git et GitHub']...


In [2]:
# %%
# 2. CRÉATION DE L'INDEX FAISS
print("\n" + "="*60)
print("🔨 CONSTRUCTION DE L'INDEX FAISS")
print("="*60)

# Initialiser le vector store
vector_store = JobVectorStore(model_name='all-mpnet-base-v2')

# Construire l'index
vector_store.build_index(jobs, index_type='flat')

# Statistiques
stats = vector_store.get_stats()
print(f"\n📊 Statistiques de l'index :")
print(f"   Offres indexées : {stats['total_jobs']}")
print(f"   Modèle : {stats['model_name']}")
print(f"   Dimensions : {stats['dimension']}")


🔨 CONSTRUCTION DE L'INDEX FAISS
✅ JobVectorStore initialisé avec all-mpnet-base-v2 (768 dimensions)

🔨 Construction de l'index FAISS...
   Nombre d'offres : 25
   Type d'index : flat
   Génération des embeddings...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Index construit avec succès !
   Total d'offres indexées : 25

📊 Statistiques de l'index :
   Offres indexées : 25
   Modèle : all-mpnet-base-v2
   Dimensions : 768


In [3]:
# %%
# 3. TEST DE RECHERCHE
print("\n" + "="*60)
print("🔍 TEST DE RECHERCHE")
print("="*60)

# Test 1 : Profil ML Engineer Junior
print("\n🧪 Test 1 : Profil ML Engineer Junior")
cv_skills_ml = ["Python", "Machine Learning", "TensorFlow", "Docker", "FastAPI", "scikit-learn"]
results_ml = vector_store.search(cv_skills_ml, top_k=5)

for i, (job, score) in enumerate(results_ml, 1):
    print(f"\n#{i} - Score FAISS : {score:.2f}%")
    print(f"   📋 {job['title']}")
    print(f"   🏢 {job['company']}")
    print(f"   📍 {job['location']}")
    print(f"   💼 Expérience : {job['experience']}")
    print(f"   🔧 Requirements : {', '.join(job['requirements'][:5])}")

# Test 2 : Profil Data Scientist
print("\n" + "-"*60)
print("🧪 Test 2 : Profil Data Scientist")
cv_skills_ds = ["Python", "R", "SQL", "Pandas", "Matplotlib", "Statistics", "A/B Testing"]
results_ds = vector_store.search(cv_skills_ds, top_k=5)

for i, (job, score) in enumerate(results_ds, 1):
    print(f"\n#{i} - Score FAISS : {score:.2f}%")
    print(f"   📋 {job['title']}")
    print(f"   🏢 {job['company']}")
    print(f"   🔧 Requirements : {', '.join(job['requirements'][:5])}")

# Test 3 : Profil DevOps
print("\n" + "-"*60)
print("🧪 Test 3 : Profil DevOps")
cv_skills_devops = ["Docker", "Kubernetes", "AWS", "Terraform", "Jenkins", "CI/CD"]
results_devops = vector_store.search(cv_skills_devops, top_k=5)

for i, (job, score) in enumerate(results_devops, 1):
    print(f"\n#{i} - Score FAISS : {score:.2f}%")
    print(f"   📋 {job['title']}")
    print(f"   🏢 {job['company']}")
    print(f"   🔧 Requirements : {', '.join(job['requirements'][:5])}")



🔍 TEST DE RECHERCHE

🧪 Test 1 : Profil ML Engineer Junior

#1 - Score FAISS : 54.41%
   📋 Junior ML Engineer
   🏢 AI Startup Paris
   📍 Toulouse, France
   💼 Expérience : 0-2 ans
   🔧 Requirements : Python (numpy, pandas, scikit-learn), Machine Learning basics (supervised learning), Git et GitHub, Docker (notions de base), Anglais technique (lecture documentation)

#2 - Score FAISS : 53.37%
   📋 MLOps Engineer
   🏢 DataCorp
   📍 Lyon, France
   💼 Expérience : 2-4 ans
   🔧 Requirements : MLOps (CI/CD pour ML), Docker, Kubernetes, Python, Cloud (AWS ou GCP), Git, GitLab/GitHub Actions

#3 - Score FAISS : 49.72%
   📋 ML Engineer - Computer Vision
   🏢 VisionTech
   📍 Paris (Hybrid)
   💼 Expérience : 1-3 ans
   🔧 Requirements : Deep Learning (CNNs, architectures modernes), PyTorch ou TensorFlow, Computer Vision (OpenCV, PIL), Python avancé, Git, Docker

#4 - Score FAISS : 47.64%
   📋 Senior ML Engineer
   🏢 BigTech France
   📍 Remote France
   💼 Expérience : 3-5 ans
   🔧 Requirements : 5+

In [4]:
# %%
# 4. ANALYSE DE LA DISTRIBUTION DES SCORES
print("\n" + "="*60)
print("📊 ANALYSE DE LA DISTRIBUTION")
print("="*60)

import numpy as np

all_scores = [score for _, score in results_ml + results_ds + results_devops]

print(f"\n📈 Statistiques des scores FAISS :")
print(f"   Moyenne : {np.mean(all_scores):.2f}%")
print(f"   Médiane : {np.median(all_scores):.2f}%")
print(f"   Min : {np.min(all_scores):.2f}%")
print(f"   Max : {np.max(all_scores):.2f}%")
print(f"   Écart-type : {np.std(all_scores):.2f}%")


📊 ANALYSE DE LA DISTRIBUTION

📈 Statistiques des scores FAISS :
   Moyenne : 41.49%
   Médiane : 44.46%
   Min : 18.61%
   Max : 60.16%
   Écart-type : 12.51%


In [ ]:
# %%
# 5. SAUVEGARDE DE L'INDEX
print("\n" + "="*60)
print("💾 SAUVEGARDE DE L'INDEX")
print("="*60)

# Créer le dossier de destination
index_dir = Path('../data/faiss_index')
index_dir.mkdir(parents=True, exist_ok=True)

# Chemins de sauvegarde
index_path = index_dir / 'jobs.index'
metadata_path = index_dir / 'jobs_metadata.pkl'

# Sauvegarder
vector_store.save(
    index_path=str(index_path),
    metadata_path=str(metadata_path)
)

print(f"\n✅ Index FAISS prêt pour l'API !")
print(f"   Index : {index_path}")
print(f"   Metadata : {metadata_path}")
print(f"   Taille index : {index_path.stat().st_size / 1024:.1f} KB")
print(f"   Taille metadata : {metadata_path.stat().st_size / 1024:.1f} KB")

# %%
# 6. VÉRIFICATION DU CHARGEMENT
print("\n" + "="*60)
print("🔄 VÉRIFICATION DU CHARGEMENT")
print("="*60)

# Créer un nouveau vector store
vector_store_test = JobVectorStore(model_name='all-mpnet-base-v2')

# Charger l'index
vector_store_test.load(
    index_path=str(index_path),
    metadata_path=str(metadata_path)
)

# Test de recherche
test_skills = ["Python", "FastAPI", "Docker"]
test_results = vector_store_test.search(test_skills, top_k=3)

print(f"\n✅ Test de recherche après chargement :")
for i, (job, score) in enumerate(test_results, 1):
    print(f"   #{i} - {job['title']} ({score:.2f}%)")

print("\n🎉 Index FAISS opérationnel !")


💾 SAUVEGARDE DE L'INDEX
✅ Index sauvegardé : ..\data\faiss_index\jobs.index
✅ Metadata sauvegardées : ..\data\faiss_index\jobs_metadata.pkl

✅ Index FAISS prêt pour l'API !
   Index : ..\data\faiss_index\jobs.index
   Metadata : ..\data\faiss_index\jobs_metadata.pkl
   Taille index : 75.0 KB
   Taille metadata : 18.3 KB

🔄 VÉRIFICATION DU CHARGEMENT
✅ JobVectorStore initialisé avec all-mpnet-base-v2 (768 dimensions)
✅ Index chargé : 25 offres
✅ Metadata chargées : 25 offres

✅ Test de recherche après chargement :
   #1 - Python Developer Junior (51.86%)
   #2 - Backend Developer Python (50.07%)
   #3 - Python Developer - Data Engineering (41.52%)

🎉 Index FAISS opérationnel !
